In [3]:
import requests

In [4]:
from bs4 import BeautifulSoup

In [15]:
from arctic import Arctic, CHUNK_STORE
lib = Arctic('localhost')
fund = lib.get_library('fund')

/home/azureuser/miniconda3/envs/fundopt/lib/python3.9/site-packages/arctic-1.80.0-py3.9.egg/arctic/store/_pandas_ndarray_store.py:8: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version


In [16]:
import pandas as pd

In [17]:
symbols = fund.list_symbols()

In [18]:
def get_fund_split_data(symbol):
    url = f'http://fundf10.eastmoney.com/fhsp_{symbol}.html'
    res = requests.get(url)
    html = BeautifulSoup(res.text)
    split_data = html.find('table', attrs={'class':'w782 comm fhxq'})
    df = pd.read_html(str(split_data))[0]
    return df

In [32]:
import tqdm

In [35]:
results = {}
for symbol in tqdm.tqdm(symbols):
    df = get_fund_split_data(symbol)
    if df.iloc[0,0] == '暂无拆分信息!':
        continue
    else:
        results[symbol] = df

100%|██████████| 3799/3799 [55:21<00:00,  1.14it/s]   


In [36]:
len(results)

364

In [37]:
results

{'000028':       年份       拆分折算日  拆分类型    拆分折算比例
 0  2016年  2016-05-31  份额折算  1:0.9985,
 '000030':       年份       拆分折算日  拆分类型    拆分折算比例
 0  2016年  2016-05-17  份额折算  1:1.8513,
 '000053':       年份       拆分折算日  拆分类型    拆分折算比例
 0  2018年  2018-12-18  份额折算  1:1.4540,
 '000058':       年份       拆分折算日  拆分类型    拆分折算比例
 0  2016年  2016-05-27  份额折算  1:1.1716,
 '000066':       年份       拆分折算日  拆分类型    拆分折算比例
 0  2016年  2016-06-07  份额折算  1:1.4113,
 '000072':       年份       拆分折算日  拆分类型    拆分折算比例
 0  2016年  2016-06-17  份额折算  1:1.7572,
 '000091':        年份       拆分折算日  拆分类型    拆分折算比例
 0   2019年  2019-01-09  份额折算  1:1.0081
 1   2018年  2018-05-09  份额折算  1:1.0080
 2   2018年  2018-05-07  份额折算  1:0.9950
 3   2018年  2018-01-09  份额折算  1:1.0090
 4   2017年  2017-05-08  份额折算  1:1.0010
 5   2017年  2017-01-09  份额折算  1:1.0070
 6   2016年  2016-09-09  份额折算  1:1.0068
 7   2016年  2016-05-09  份额折算  1:1.0050
 8   2016年  2016-05-05  份额折算  1:1.0030
 9   2016年  2016-01-08  份额折算  1:1.0059
 10  2015年  2015-09-09  份额折算  1:1.0102
